In [ ]:
import os

from scipy.stats import poisson
from statsmodels.genmod.generalized_linear_model import GLMResultsWrapper
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
plt.rcParams["figure.figsize"] = 12/2.54, 8/2.54

## Distribution of shots and goals

First, let us use WyScout data to plot the distribution of shots and goals in a Bundesliga season. Let us first load the data.

In [ ]:
data_dir: str = "../../data/wyscout/events"

In [ ]:
data: pd.DataFrame = pd.read_json(os.path.join(data_dir, "events_Germany.json"))
data.head()

In [ ]:
data.shape

Next, let us add an indicator column that specifies if the event (i.e. row) refers to a shot on goal.

In [ ]:
shot_event_names: list = ["Shot", "Free kick shot", "Penalty"]

In [ ]:
shots = data[data["subEventName"].isin(shot_event_names)]
shots.head()

We need another column that specifies if a shot resulted in a goal. This information is present in the `tags` column. If there was a goal scored from a shot, it contains a dictionary with a value `101` for the `id` key. Let us create this column.

In [ ]:
def is_shot_id_present(tags: list) -> int:
    tag: dict
    for tag in tags:
        if tag["id"] == 101:
            return 1
    return 0

In [ ]:
shots.loc[:, "goal"] = shots.apply(lambda x: is_shot_id_present(x["tags"]), axis=1)
shots.head()

As we want to visualise the distribution of shots and goals over an entire season, let us aggregate them by match.

In [ ]:
shots_per_match: list = []
goals_per_match: list = []
match_id: str
match_data: pd.DataFrame
for match_id, match_data in shots.groupby("matchId"):
    shots_per_match.append(len(match_data))
    goals_per_match.append(sum(match_data["goal"]))

First, let us create a histogram of the goals.

In [ ]:
mean_goals: float = np.mean(goals_per_match)
goals_dist: np.ndarray
goals_bins: np.ndarray
goals_dist, goals_bins = np.histogram(goals_per_match, bins=np.arange(-0.5, 10.5))
goals_dist = goals_dist / shots["matchId"].nunique()

Based on the mean goals, let us create a Poisson distribution.

In [ ]:
possible_goals: np.ndarray = np.arange(0, 10).astype(int)
x: int
possible_goals_factorial: np.ndarray = np.array([np.math.factorial(x) for x in possible_goals])
poisson_possible_goals: np.ndarray = np.power(mean_goals, possible_goals) * np.exp(-mean_goals) / possible_goals_factorial

Finally, let us plot the distribution of goals in the Bundesliga season.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

plt.hist(possible_goals - 0.5, 9, weights=goals_dist)
plt.plot(possible_goals, poisson_possible_goals, color="black")
ax.set_yticks(np.arange(0, 0.3, 0.1))
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_position("zero")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticks(np.arange(0, 10, 1))
ax.set_ylabel("Proportion of matches")
ax.set_xlabel("Number of goals scored")
plt.show()


#Exercise:
#1, Make a histogram of shots per game
#2, Find the mean and standard deviation for shots per game
#3, Show that shots per game is roughly normally distributed.

Some observations from the plot above are:
- The Poisson distribution under-represents games with 2 goals (2-0, 0-2, or 1-1) while it over-represents single goal games.

Let us now recreate the charts for shots per game. As there are way more shots per game than the number of goals, the Binomial distribution, which is the basis, converges to Normal distribution under the Law of Large Numbers. So, just like we plotted the Poisson distribution (as a Black line) with the mean number of goals per game as the parameters, we will need the mean number of shots per game and the standard deviation as parameters to plot the Normal distribution.

In [ ]:
mean_shots: float = np.mean(shots_per_match)
std_shots: float = np.std(shots_per_match)
shots_dist: np.ndarray
shots_bins: np.ndarray
shots_dist, shots_bins = np.histogram(shots_per_match, bins=np.arange(-0.5, 50.5))
shots_dist = shots_dist / shots["matchId"].nunique()

In [ ]:
possible_shots: np.ndarray = np.arange(0, 50).astype(int)
x: int
normal_possible_shots: np.ndarray = np.array([scipy.stats.norm(mean_shots, std_shots).pdf(x) for x in possible_shots])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

plt.hist(possible_shots - 0.5, 49, weights=shots_dist)
plt.plot(possible_shots, normal_possible_shots, color="black")
ax.set_yticks(np.arange(0, 0.15, 0.05))
ax.spines["left"].set_visible(True)
ax.spines["bottom"].set_position("zero")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.set_xticks(np.arange(0, 50, 5))
ax.set_ylabel("Proportion of matches")
ax.set_xlabel("Number of shots hit")
plt.show()


Some observations from the chart above:
- There seem to be at least 10 shots per game and at most 39.
- As the mean of the distribution suggests, there are 20-25 shots taken per game.

## Simulate a season

Let us now see how to simulate matches over an entire season based on the league table for the previous season. We begin by download the table for the 2019-20 season of the English Premier League.

In [ ]:
epl: pd.DataFrame = pd.read_csv("https://www.football-data.co.uk/mmz4281/1920/E0.csv")
epl.head()

Let us select the required columns and name them according to our preferences.

In [ ]:
epl = epl.loc[:, ["HomeTeam", "AwayTeam", "FTHG", "FTAG"]].rename(columns={"FTHG": "HomeGoals", "FTAG": "AwayGoals"})
epl.head()

Next, we want to fit a Poisson model that estimates the numbers of goals that will be scored by a team given the fixed effects like the base rate of the team itself, the opponent, and whether the game is at home or away. This is also called the *Dickson-Coles Model*. To fit this model, let us first construct the data.

In [ ]:
goal_model_data: pd.DataFrame = pd.concat([
    (epl[["HomeTeam", "AwayTeam", "HomeGoals"]].assign(home=1)
     .rename(columns={"HomeTeam": "team", "AwayTeam": "opponent", "HomeGoals": "goals"})),
    (epl[["AwayTeam", "HomeTeam", "AwayGoals"]].assign(home=0)
     .rename(columns={"AwayTeam": "team", "HomeTeam": "opponent", "AwayGoals": "goals"}))
])
goal_model_data.head()

Now we fit the model using the `statsmodels` package.

In [ ]:
poisson_model: GLMResultsWrapper = smf.glm(formula="goals ~ home + team + opponent",
                                           data=goal_model_data,
                                           family=sm.families.Poisson()).fit()
poisson_model.summary()

In the result above, the *intercept* refers to the first team in our data i.e. Arsenal. Some observations from the model summary are:
- As the y-variable is the number of goals scored, coefficients of the home team (starting with `team`) represents how many more or fewer goals they score as compared to Arsenal per game. We see that only a handful of teams like Manchester City, Liverpool do better on this metric than Arsenal.
- The coefficients of the away team (starting with `opponent`) indicate how many goals more does a team concede per game as compared to Arsenal.
- We need to pay attention to the significance of these coefficients i.e. the $P > |z|$ column. A coefficient is said to be statistically significant if its p-value is less than `0.05`. When we consider the P-value, we see that only Manchester City and Liverpool score statistically significant more goals than Arsenal. Watford, Norwich, and Crystal Palace. We cannot make the same argument for any team conceding more goals than Arsenal.
- Regardless of the fixed effects, we can see that playing at home carries a significant advantage in terms of the number of goals scored.

Let us now consider the home team to be Manchester City and the away team to the Arsenal. We can use the model to predict the goals scored by each team in this case.

In [ ]:
home_team: str = "Man City"
away_team: str = "Arsenal"

In [ ]:
home_score_rate: pd.DataFrame = poisson_model.predict(
    pd.DataFrame(
        data={
            "team": home_team,
            "opponent": away_team,
            "home": 1
        },
        index=[1]
    )
)
away_score_rate: pd.DataFrame = poisson_model.predict(
    pd.DataFrame(
        data={
            "team": away_team,
            "opponent": home_team,
            "home": 0
        },
        index=[1]
    )
)
print(f"{home_team} against {away_team} expect to score: {round(home_score_rate.loc[1], 2)}")
print(f"{away_team} against {home_team} expect to score: {round(away_score_rate.loc[1], 2)}")

Given that we derived the results from a Poisson model, the predictions above can be considered as the mean of the distribution. We can then simulate multiple matches between these two teams by providing this mean to the Poisson distribution.

Let us write a function to do so. In this function, we will compute two lists, one each for the home and away team. The list represents the probability of the team scoring $i$ goals, where $i$ is the index (starting at 0), given the probability derived from our Poisson model. We then compute an outer matrix product of the two lists to yield the probabilities of various scorelines like 0-0, 1-0, etc.

In [ ]:
def simulate_matches(
        goal_model: GLMResultsWrapper,
        home_side: str,
        away_side: str,
        max_goals: int = 10
):
    mean_home_goals: float = goal_model.predict(
        pd.DataFrame(
            data={
                "team": home_side,
                "opponent": away_side,
                "home": 1
            },
            index=[1]
        )
    ).loc[1]
    mean_away_goals: float = goal_model.predict(
        pd.DataFrame(
            data={
                "team": away_side,
                "opponent": home_side,
                "home": 0
            },
            index=[1]
        )
    ).loc[1]

    i: int
    team_mean: tuple
    # Given the mean scoring rate of a team, compute the probability that a team will
    # score `i` goals.
    simulated_goals: list = [[poisson.pmf(i, team_mean)
                              for i in range(0, (max_goals + 1))]
                              for team_mean in [mean_home_goals, mean_away_goals]]

    return np.outer(np.array(simulated_goals[0]), np.array(simulated_goals[1]))

In [ ]:
required_max_goals: int = 5
score_matrix: np.ndarray = simulate_matches(poisson_model, home_team, away_team, max_goals=required_max_goals)
score_matrix

In the table above, the rows represent the number of goals scored by Manchester City and the column represent goals by Arsenal. We see that the 2-0 scoreline has the highest probability.

Instead of focusing further on the numbers, let us create a heatmap of these probabilities to make it easier to read the results.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

pos=ax.imshow(score_matrix,
              aspect="auto",
              cmap=plt.cm.Reds)
fig.colorbar(pos, ax=ax)
ax.set_title("Probability of outcome")
plt.xlim((-0.5, 5.5))
plt.ylim((-0.5, 5.5))
plt.tight_layout()
ax.set_xlabel(f"Goals scored by {away_team}")
ax.set_ylabel(f"Goals scored by {home_team}")
plt.show()

We can see that our model mostly predicts a win for Manchester city with them scoring 2-3 goals and Arsenal scoring 1-2 goals.